First section - find all canadian FRC teams, and get a list of the unique postal codes to send to geocod.io in order to get census division ids for them
--

In [1]:
import pandas as pd
import tbapy
tba = tbapy.TBA('kuosW2jPtJ4Lj0m3HxcDr9mLTI7JHSzimVTYvBrbZdtFdMKqPwzDpnE4jTZy9G6G')
file_dest = "/home/will/Documents/Projects/frc_counties/data/zipcode_request.csv"

In [2]:
# find all canada teams
teams = [team for team in tba.teams(year=2020) if team.country == "Canada"]
for i in range(min(len(teams), 5)):
    print(teams[i].key[3:], ",", teams[i].postal_code)

188 , M1G 3M3
296 , H4B 1R2
610 , m2l1a2
746 , M3N 1W7
771 , L6J 2L1


In [3]:
# print first 50 unique frc team zip codes
teams_postalcodes = [team.postal_code.upper() for team in teams]
print(len(teams_postalcodes))
teams_postalcodes = list(set(teams_postalcodes))
print(len(teams_postalcodes))
print(set(teams_postalcodes[:min(50,len(teams_postalcodes))]))

270
267
{'H3P 2X8', 'H3H 1E3', 'P3E 2P8', 'K2C3M5', 'H2P 1W9', 'H1N 1A2', 'H2E 1R9', 'N8X 2R7', 'L2N 2Z7', 'G5A 1T3', 'M6A 1B1', 'N5Y 5R6', 'P0P 2J0', 'P1L 2E9', 'V2A 3T1', 'L5R 3N9 ', 'V3H 2M5', 'V6L 2E7', 'L5N3B2', 'L4L1J3', 'M9B 3G7', 'P7A2J6', 'L4G1J5', 'N6H 1V4', 'N2G 3C8', 'M2N 7H1', 'T1J3E5', 'L8W 1B8', 'K7S 1X2', 'J2B6A2', 'M4M 2A1', 'J1H 1L6', 'N0M 2K0', 'N2H 1H6', 'K7L2Z5', 'L3P 4N3', 'M6R 2V5', 'N1L 1G1', 'L1R 2K5', 'N6C1T4', 'H2K 3S9', 'N5X 4B1', 'G5Y 5S1', 'M9V 3B2', 'L6S 1V3', 'T3J 0H5', 'J2K 3R9', 'G8Z 3R8', 'L8E 4M5', 'L8S 4P3'}


In [4]:
# generate csv to send into geocod.io
to_send_str = "Zip,Country\n"
for code in teams_postalcodes:
    to_send_str += code + ",Canada\n"
writer = open(file_dest, "w")
writer.write(to_send_str)
writer.close()

Now go and run zipcode_request through Geocod.io, and save it as canadian_zipcodes_to_cd.csv in the data folder.  
--
Then continue on to make a dataset for FRC Team -> Census Division ID
--

In [5]:
file_src = "/home/will/Documents/Projects/frc_counties/data/canadian_zipcodes_to_cd.csv"
file_dest = "/home/will/Documents/Projects/frc_counties/data/canadian_team_to_cd_id.csv"

In [6]:
# read in zipcode request
zipcode_data = pd.read_csv(file_src)
print(zipcode_data.columns)

Index(['Zip', 'Country', 'Latitude', 'Longitude', 'Accuracy Score',
       'Accuracy Type', 'Number', 'Street', 'City', 'State', 'County', 'Zip.1',
       'Country.1', 'Source', 'Statcan Census Division id',
       'Statcan Census Division Name', 'Statcan Census Division Type',
       'Statcan Census Division Type Description',
       'Statcan Census Consolidated Subdivision id',
       'Statcan Census Consolidated Subdivision Name',
       'Statcan Census Subdivision id', 'Statcan Census Subdivision Name',
       'Statcan Census Subdivision Type',
       'Statcan Census Subdivision Type Description',
       'Statcan Census Economic Region',
       'Statcan Census Statistical Area Code',
       'Statcan Census Statistical Area Code Description',
       'Statcan Census Statistical Area Type',
       'Statcan Census Statistical Area Type Description',
       'Statcan Census CMA/CA id', 'Statcan Census CMA/CA Name',
       'Statcan Census CMA/CA Type', 'Statcan Census CMA/CA Type Descript

In [7]:
# row in merged_data = (team, statcan census division id)
def getStatcanCID(zipcode):
    for idx, row in zipcode_data.iterrows():
        if row["Zip"] == zipcode:
            return row["Statcan Census Division id"]

merged_data = [[team, None] for team in teams]
for idx, [team, b] in enumerate(merged_data):
    merged_data[idx][1] = getStatcanCID(team.postal_code.upper())
    if idx < 5:
        print(team.key, merged_data[idx][1])

frc188 3520
frc296 2466
frc610 3520
frc746 3520
frc771 3524


In [8]:
#  write to file
to_write = "Team,cd_id\n"
for [team, cd_id] in merged_data:
    to_write += team.key[3:] + "," + str(cd_id) + "\n"

writer = open(file_dest, "w")
writer.write(to_write)
writer.close()

Now let's append to counties_loc (county id -> lat/long) dataset our (census division id -> lat/long) data. Use the first column "Sort [1]" and set to -1 for all canada rows. We'll use this in our program to differentiate whether to parse the id as a USA county id or a Canadian county ID
--

In [9]:
counties_loc = "/home/will/Documents/Projects/frc_counties/data/counties_loc.csv"
counties_app_loc = "/home/will/Documents/Projects/frc_counties/data/counties_loc_w_canada.csv"

In [10]:
# get data in good format
def getLatLong(zipcode):
    for idx, row in zipcode_data.iterrows():
        if row["Zip"] == zipcode:
            return row["Latitude"], row["Longitude"]

        
# merged_data row = (zipcode, lat, long)
merged_data = [[code, None, None] for code in teams_postalcodes]
for idx, [code, b, c] in enumerate(merged_data):
    lat, long = getLatLong(code)
    merged_data[idx][1] = lat
    merged_data[idx][2] = long
    if idx < 5:
        print(code, "|", merged_data[idx][1], merged_data[idx][2])

H3P 2X8 | 45.5217 -73.6393
P3E 2P8 | 46.49272 -80.99121099999999
H3H 1E3 | 45.5009 -73.5877
K2C3M5 | 45.3679 -75.7381
H2P 1W9 | 45.5435 -73.6339


In [11]:
# load counties_loc into pd
cnt_loc = pd.read_csv(counties_loc)
print(len(cnt_loc.index))
cnt_loc.head()

3144


,Sort [1],State,FIPS,County [2],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,c01001,Autauga,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
1,2,AL,c01003,Baldwin,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
2,3,AL,c01005,Barbour,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
3,4,AL,c01007,Bibb,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
4,5,AL,c01009,Blount,"57,322","1,669.96",644.776,15.157,5.852,"1,685.12",650.628,33.977448,-86.567246


In [12]:
# append canada
for code, lat, long in merged_data:
    cnt_loc = cnt_loc.append(pd.Series([-1, None, code, None, None, None, None, None, None, None, None, lat, long], index = cnt_loc.columns), ignore_index=True)
print(len(cnt_loc.index))
cnt_loc.tail()

3411


,Sort [1],State,FIPS,County [2],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
3406,-1,None,T0L1Y0,None,None,None,None,None,None,None,None,51.424967,-116.177535
3407,-1,None,V3N 5E5,None,None,None,None,None,None,None,None,49.227500,-122.930100
3408,-1,None,V7M 2W7,None,None,None,None,None,None,None,None,49.320713,-123.073783
3409,-1,None,K2A 0E2,None,None,None,None,None,None,None,None,45.380500,-75.763600
3410,-1,None,L1J 1G5,None,None,None,None,None,None,None,None,43.890650,-78.892768


In [13]:
# write csv
cnt_loc.to_csv(counties_app_loc)